---

## Next Steps

The following cells will be filled in as each framework feature is implemented:

- Models demonstration  
- Prompt renderer example  
- Agent planning  
- Agent run (with MockLLM)  
- EvalResults display

Each cell will be marked with a comment like `# TASK-3: renderer example` for traceability.


In [1]:
# TASK-7.1: Import core framework classes (deferred until implemented)
# from research_agent_framework.agents.base import ResearchAgent
# from research_agent_framework.llm.client import MockLLM
# from research_agent_framework.adapters.search import MockSearchAdapter


In [2]:
# TASK-7.1: Bootstrap environment
# from research_agent_framework.bootstrap import bootstrap
# bootstrap()


In [3]:
# TASK-7.1: sys.path fix for local imports
import sys
sys.path.append('../src')  # Adjust path as needed for local package imports


# Consolidated Research Agent Demo

This notebook demonstrates the use of the `research_agent_framework` package. Each section will be updated as new features are implemented.

---


# Consolidated Research Agent Demo Notebook

This notebook demonstrates the use of the `research_agent_framework` package and its components. Each section is marked for traceability to the corresponding PRD task.

---


In [4]:
# TASK-7.1: sys.path fix for local imports
import sys
from pathlib import Path
sys.path.insert(0, str(Path('..').resolve() / 'src'))


In [5]:
# TASK-7.1: Bootstrap environment
from research_agent_framework.bootstrap import bootstrap
bootstrap()


In [6]:
# TASK-2.3: models demo
from research_agent_framework.config import Settings, get_settings
from research_agent_framework.models import Scope, ResearchTask, EvalResult, SerpResult
from pydantic import TypeAdapter, HttpUrl
from assertpy import assert_that

# Construct model instances
scope = Scope(topic='Coffee Shops', description='Find coffee shops in SF', constraints=['no paid sources'])
task = ResearchTask(id='t-001', query='best coffee in soma')
eval_result = EvalResult(task_id=task.id, success=True, score=0.95, feedback='Looks good')
# Use TypeAdapter to validate/construct an HttpUrl (pydantic v2)
url_adapter = TypeAdapter(HttpUrl)
validated_url = url_adapter.validate_python('https://example.com')
serp = SerpResult(title='Cafe Example', url=validated_url, snippet='Great coffee', raw={'id': 1})

# Example asserts using assertpy
assert_that(scope.topic).is_equal_to('Coffee Shops')
assert_that(scope.constraints).contains('no paid sources')
assert_that(task.id).is_equal_to('t-001')
assert_that(eval_result.success).is_true()
assert_that(serp.url).is_instance_of(HttpUrl)

from rich.console import Console
from typing import cast
console =get_settings().console
assert_that(console).is_not_none()
console = cast(Console, console)
console.print(f"{scope=}")
console.print(f"{task=}")
console.print(f"{eval_result=}")
console.print(f"{serp=}")


scope=Scope(topic='Coffee Shops', description='Find coffee shops in SF', constraints=['no paid sources'])

task=ResearchTask(id='t-001', query='best coffee in soma', context={}, notes=None)

eval_result=EvalResult(task_id='t-001', success=True, score=0.95, feedback='Looks good', details={})

serp=SerpResult(title='Cafe Example', url=HttpUrl('https://example.com/'), snippet='Great coffee', raw={'id': 1})

In [7]:
# TASK-7.1: Import framework components (deferred until implemented)
# from research_agent_framework.agents.base import ResearchAgent
# from research_agent_framework.llm.client import MockLLM
# from research_agent_framework.adapters.search import MockSearchAdapter
